**Step1:** Create and Setup Hugging Face API Access Token Key




**Step2:** Install required dependencies

In [2]:
# Step2: Install required dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl==0.14.0 peft==0.14.0 xformers==0.0.28.post3
!pip install torch==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-bz8mrh1t/unsloth_96c759da54e24fa09bff3be064696c4c
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-bz8mrh1t/unsloth_96c759da54e24fa09bff3be064696c4c
  Resolved https://github.com/unslothai/unsloth.git to commit eaeba82aee888324ee2c9839939401ce32fc63ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: trl
    Found existing installation: trl 0.19.0
    Uninstalling trl-0.19.0:
      Successfully uninstalled trl-0.19.0
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled pef

In [4]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import notebook_login, login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN2')
login(hf_token)

In [6]:
# Check GPU availability
import torch

print("Device Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA is not available." )

Device Name:  Tesla T4


**Step:** Setup a pretrained DeepSeek-R1

In [34]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length=max_sequence_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.6.12: Fast Llama patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [96]:
train_prompt_style = """
You are a friendly and helpful medical assistant chatbot. Provide clear and simple explanations about common health questions and symptoms. Always remind users that your advice is general and does not replace a visit to a doctor. Encourage users to seek professional medical help for serious or urgent issues.

### Query:
{}

### Answer:
<think>{}
"""


In [97]:
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")


In [ ]:
# Run Inference on the model

# Define a test question
question = """What could be the cause of a persistent cough lasting more than two weeks?"""

# Tokenize the Input
inputs = tokenizer([train_prompt_style.format(question, "", "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True,
    # eos_token_id = tokenizer.eos_token_id,
    # pad_token_id = tokenizer.eos_token_id,
)


# Decode the response tokens back to text

response = tokenizer.batch_decode(outputs)
print(response)

In [15]:
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split= "train[:500]", trust_remote_code = True)



In [16]:
# medical_dataset[0]

In [42]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [65]:
#prepare fint-tuning
def preprocess_input_data(examples):
  inputs = examples["Question"]
  cot = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []
  for input, cot, output in zip(inputs, cot, outputs):
    text = train_prompt_style.format(input, cot, outputs) + EOS_TOKEN
    texts.append(text)


  return {"text": texts}

In [66]:
finetune_dataset = medical_dataset.map(preprocess_input_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [68]:
finetune_dataset["text"][0]

'\nYou are a knowledgeable and empathetic medical assistant chatbot. \nYour role is to provide clear, accurate, and easy-to-understand information about common medical conditions, symptoms, treatments, and general health advice. \nAlways remind users that your responses do not replace professional medical diagnosis or treatment, \nand encourage them to consult a healthcare provider for serious or urgent issues.\n\n### Instruction:\n- Use simple, non-technical language suitable for a general audience.\n- Be empathetic and supportive in tone.\n- Provide evidence-based information based on current medical knowledge.\n- Avoid giving specific prescriptions or diagnoses.\n- Encourage users to seek immediate medical attention if symptoms are severe or life-threatening.\n- If a question is outside your scope, politely inform the user and suggest consulting a professional.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the prese

In [69]:
model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

In [89]:
if hasattr(model, 'unwrapped_old_generate'):
  del model.unwrapped_old_generate

In [70]:
trainer = SFTTrainer(
  model = model_lora,
  tokenizer = tokenizer,
  train_dataset = finetune_dataset,
  dataset_text_field = "text",
  max_seq_length = max_sequence_length,
  dataset_num_proc = 1,


  # Define training args
  args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    num_train_epochs = 1,
    max_steps = 60,
    learning_rate = 2e-4,
    fp16 = not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir = "outputs",
  ),
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [75]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
import wandb
wandb.login(key=wnb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [76]:
# Start training the model
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.814900
20,0.934100
30,0.868200
40,0.824700
50,0.868600
60,0.855400


TrainOutput(global_step=60, training_loss=1.0276502609252929, metrics={'train_runtime': 1447.2282, 'train_samples_per_second': 0.332, 'train_steps_per_second': 0.041, 'total_flos': 2.225661850681344e+16, 'train_loss': 1.0276502609252929, 'epoch': 0.96})

In [77]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▂▁▁▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▁▁▁▁
total_flos,2.225661850681344e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.19325
train/learning_rate,0.0
train/loss,0.8554


In [ ]:
# Run Inference on the model

# Define a test question
question = """What could be the cause of a persistent cough lasting more than two weeks?"""
FastLanguageModel.for_inference(model_lora)
# Tokenize the Input
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate(
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True,
    # eos_token_id = tokenizer.eos_token_id,
    # pad_token_id = tokenizer.eos_token_id,
)


# Decode the response tokens back to text

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Answer:")[1])

In [ ]:
print(response)